# 🚀 Proyecto Final: Análisis de Propiedades de Airbnb

**Autor:** Erick Catalán
**Fecha:** 27 de Noviembre de 2025



In [23]:
## 1. Importación de Librerías 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Librerías de ML necesarias:
from sklearn.model_selection import train_test_split, RandomizedSearchCV # RandomizedSearchCV importado
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import time

In [24]:
## 2. Carga Inicial del Dataset


df = pd.read_csv('listings.csv')

print("--- Dimensiones del Dataset ---")
print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}") 
print(f"¡El dataset tiene {df.shape[0]} filas, cumpliendo el requisito de 3,000+!")

--- Dimensiones del Dataset ---
Filas: 26401, Columnas: 79
¡El dataset tiene 26401 filas, cumpliendo el requisito de 3,000+!


In [25]:
## 3. Inspección Inicial de Datos

print("\n--- Primeras 5 Filas ---")
display(df.head())

print("\n--- Tipos de Datos y Resumen de Nulos ---")
df.info()


--- Primeras 5 Filas ---


,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,35797,https://www.airbnb.com/rooms/35797,20250625031918,2025-06-26,city scrape,Villa Dante,"Dentro de Villa un estudio de arte con futon, ...","Santa Fe Shopping Mall, Interlomas Park and th...",https://a0.muscache.com/pictures/f395ab78-1185...,153786,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
1,44616,https://www.airbnb.com/rooms/44616,20250625031918,2025-07-01,city scrape,Condesa Haus,A new concept of hosting in mexico through a b...,NaN,https://a0.muscache.com/pictures/251410/ec75fe...,196253,...,4.78,4.98,4.47,NaN,f,9,4,2,0,0.39
2,56074,https://www.airbnb.com/rooms/56074,20250625031918,2025-07-01,city scrape,Great space in historical San Rafael,This great apartment is located in one of the ...,Very traditional neighborhood with all service...,https://a0.muscache.com/pictures/3005118/60dac...,265650,...,4.94,4.76,4.79,NaN,f,1,1,0,0,0.48
3,67703,https://www.airbnb.com/rooms/67703,20250625031918,2025-07-01,city scrape,"2 bedroom apt. deco bldg, Condesa","Comfortably furnished, sunny, 2 bedroom apt., ...",NaN,https://a0.muscache.com/pictures/3281720/6f078...,334451,...,4.92,4.98,4.92,NaN,f,2,2,0,0,0.30
4,70644,https://www.airbnb.com/rooms/70644,20250625031918,2025-07-01,city scrape,Beautiful light Studio Coyoacan- full equipped !,COYOACAN designer studio quiet & safe! well eq...,Coyoacan is a beautiful neighborhood famous fo...,https://a0.muscache.com/pictures/f397d2da-d045...,212109,...,4.98,4.96,4.92,NaN,f,3,2,1,0,0.81



--- Tipos de Datos y Resumen de Nulos ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26401 entries, 0 to 26400
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            26401 non-null  int64  
 1   listing_url                                   26401 non-null  object 
 2   scrape_id                                     26401 non-null  int64  
 3   last_scraped                                  26401 non-null  object 
 4   source                                        26401 non-null  object 
 5   name                                          26401 non-null  object 
 6   description                                   25633 non-null  object 
 7   neighborhood_overview                         13970 non-null  object 
 8   picture_url                                   26401 non-null  object 
 9   host_id           

In [26]:
## 4. Limpieza de Estructura: Eliminación de Duplicados

print("--- Diagnóstico de Filas Duplicadas ---")
print(f"Filas iniciales antes de la limpieza: {df.shape[0]}")

# 5.1. Identificar y contar las filas completamente idénticas
duplicate_rows = df.duplicated().sum()
print(f"Filas duplicadas encontradas: {duplicate_rows}")

if duplicate_rows > 0:
    # 5.2. Eliminar los duplicados, manteniendo la primera aparición
    df.drop_duplicates(inplace=True)
    print("¡Duplicados eliminados exitosamente!")
else:
    print("No se encontraron filas completamente duplicadas.")

print(f"Filas después de la limpieza de duplicados: {df.shape[0]}")

--- Diagnóstico de Filas Duplicadas ---
Filas iniciales antes de la limpieza: 26401
Filas duplicadas encontradas: 0
No se encontraron filas completamente duplicadas.
Filas después de la limpieza de duplicados: 26401


## 5. Contexto del Problema y Preguntas de Investigación (Actualizado)

El objetivo principal de este proyecto es aplicar técnicas de Ciencia de Datos al dataset de **Anuncios de Airbnb (listings.csv)** para identificar las características clave que impulsan el **máximo POTENCIAL DE INGRESO NETO / RENTABILIDAD**.

Para lograr esto, desarrollamos una métrica *proxy* de rentabilidad que va más allá del simple precio, integrando **Ingreso Potencial** y **Calidad Operacional/Demanda**.

---

### Variable Objetivo Clave: Rentabilidad Proxy Potenciada

Dado que no tenemos datos de costos operacionales, hemos creado un **índice de Rentabilidad** que combina tres factores críticos para un listing de éxito:

1.  **Potencial de Ingreso (Precio):** El valor monetario base.
2.  **Calidad Operacional (Puntuación de Reseñas):** La satisfacción del cliente y la capacidad de mantener precios altos a largo plazo.
3.  **Demanda Sostenida (Tasa de Ocupación - *Estimated Occupancy*):** El uso real del listing, que es un factor directo de ingreso y demanda.

Hemos definido el *Target* del modelo como:

$$\text{Rentabilidad Proxy} = \\text{Precio} \\times \\text{Puntuación de Reseñas} \\times \\text{Tasa de Ocupación}$$

Y para estabilizar la distribución y mejorar la predicción del modelo de regresión, aplicamos una transformación logarítmica:

$$\text{Target del Modelo} = \\mathbf{\log(\text{Rentabilidad Proxy} + 1)}$$

### Preguntas Clave (Centradas en la Rentabilidad Potenciada)

1.  **Pregunta de Predicción (para el Modelo ML):** ¿Cuáles son los **predictores más significativos** de la **Rentabilidad Proxy $\log$** de una propiedad? (La respuesta a esto se encuentra en el Análisis de Importancia de Características, Celda 13).
    * *Modelo Usado: **Regresión (Random Forest)** para predecir el valor de la Rentabilidad Proxy $\log$.*
2.  **Pregunta de Inversión Accionable:** ¿Qué **criterios de inversión específicos** (características del host, tipo de propiedad, ubicación) debe usar un inversor para maximizar su **Rentabilidad** en propiedades nuevas?
3.  **Pregunta de Negocio Operacional:** ¿Cómo se relacionan los factores de **Madurez Operacional** (ej. `number_of_reviews`) y **Gestión** (ej. `calculated_host_listings_count`) con la Rentabilidad más allá del factor Ubicación?

In [27]:
## 6. Manejo de Valores Nulos y Corrección de Tipos (Misión 1 Finalizada)

# 6.1. Eliminar columnas con 100% de nulos
cols_to_drop = ['neighbourhood_group_cleansed', 'calendar_updated', 'license']
df.drop(columns=cols_to_drop, inplace=True, errors='ignore')
print(f"Columnas con 100% de nulos eliminadas: {cols_to_drop}")

# 6.2. Conversión y Limpieza de PRICE
if df['price'].dtype == 'object':
    df['price'] = df['price'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False).astype(float)
    print("Columna 'price' convertida a float.")

# 6.3. Imputar nulos en Puntuación de Reseñas (Crítico para la Rentabilidad Proxy)
# Usaremos la mediana para no afectar la distribución de scores.
median_score = df['review_scores_rating'].median()
df['review_scores_rating'].fillna(median_score, inplace=True)
print(f"Nulos imputados en 'review_scores_rating' con la mediana: {median_score}")

# 6.4. Limpieza final de filas críticas (price y scores)
df.dropna(subset=['price', 'review_scores_rating'], inplace=True)

print(f"\nFilas finales después del tratamiento de nulos: {df.shape[0]}")

Columnas con 100% de nulos eliminadas: ['neighbourhood_group_cleansed', 'calendar_updated', 'license']
Columna 'price' convertida a float.
Nulos imputados en 'review_scores_rating' con la mediana: 4.84

Filas finales después del tratamiento de nulos: 23127


/var/folders/10/fcvqgkms1m3563dl4f2_c33w0000gn/T/ipykernel_61183/4253367353.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['review_scores_rating'].fillna(median_score, inplace=True)


In [28]:
## 7. Creación de la Variable Proxy de Rentabilidad (Net Income Potential)

# 7.1. Ingeniería de la Tasa de Ocupación (Proxy de Demanda Sostenida)
df['occupancy_rate'] = df['estimated_occupancy_l365d'] / 365

# 7.2. Crear la variable de Rentabilidad Proxy: Precio x Puntuación de Reseñas x Tasa de Ocupación
df['rentability_proxy'] = df['price'] * df['review_scores_rating'] * df['occupancy_rate']

# 7.3. Aplicar la Transformación Logarítmica 
df['log_rentability'] = np.log1p(df['rentability_proxy'])

print("Variable 'log_rentability' Creada y Transformada (incluyendo Tasa de Ocupación).")

Variable 'log_rentability' Creada y Transformada (incluyendo Tasa de Ocupación).


In [37]:
## 8. Manejo de Outliers  (Clipping) 


lower_bound = df['log_rentability'].quantile(0.01)
upper_bound = df['log_rentability'].quantile(0.99)

df['log_rentability_clipped'] = np.clip(
    df['log_rentability'], 
    a_min=lower_bound, 
    a_max=upper_bound
)


df['log_rentability'] = df['log_rentability_clipped']

print(f"Límite Inferior (1%): {lower_bound:.4f}, Límite Superior (99%): {upper_bound:.4f}")
print("¡Outliers de 'log_rentability' recortados! El RMSE será estable.")

# Limpiamos columnas auxiliares
df.drop(columns=['log_rentability_clipped', 'occupancy_rate'], inplace=True, errors='ignore')

Límite Inferior (1%): 0.0000, Límite Superior (99%): 9.4325
¡Outliers de 'log_rentability' recortados! El RMSE será estable.


In [38]:
## 9. Análisis de Correlación con Rentabilidad (EDA Avanzado)

key_vars = [
    'log_rentability', 'accommodates', 'minimum_nights', 'number_of_reviews', 
    'host_listings_count', 'review_scores_rating', 'estimated_occupancy_l365d'
]
corr_df = df[key_vars].copy()

correlation_matrix = corr_df.corr()

# Identificar los 5 mejores predictores de Log Rentabilidad
rentability_corr = correlation_matrix['log_rentability'].sort_values(ascending=False).drop('log_rentability')

print("--- Top 5 Correlaciones Numéricas (Target TRUNCADO) ---")
display(rentability_corr.head(5))

--- Top 5 Correlaciones Numéricas (Target TRUNCADO) ---


estimated_occupancy_l365d    0.736602
number_of_reviews            0.420629
accommodates                 0.191075
host_listings_count          0.119866
review_scores_rating         0.086972
Name: log_rentability, dtype: float64

In [39]:
## 10. Nivel 3 - Preparación de Datos: Feature Engineering y Selección

# 10.1. Ingeniería de Características Categóricas Clave (Target Encoding)
rentability_mean_by_neighbourhood = df.groupby('neighbourhood_cleansed')['log_rentability'].mean()
df['neighbourhood_rentability_mean'] = df['neighbourhood_cleansed'].map(rentability_mean_by_neighbourhood)

# 10.2. Definir la Variable Objetivo (Target)
TARGET_VARIABLE = 'log_rentability'
y = df[TARGET_VARIABLE]

# 10.3. Seleccionar las Características (Features)
X = df[[
    'accommodates', 
    'review_scores_rating', 
    'neighbourhood_rentability_mean', # FEATURE ENCODEADA
    'calculated_host_listings_count',
    'host_is_superhost', 
    'room_type', 
    'property_type',
    'minimum_nights',
    'number_of_reviews' # AÑADIDO POR CORRELACIÓN
]].copy()

# 10.4. Identificar tipos de variables para el preprocesamiento
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include='object').columns.tolist()

print(f"Número de características (features) seleccionadas: {X.shape[1]}")

Número de características (features) seleccionadas: 9


In [ ]:
## 11. Codificación y Escalado 

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 11.1. Definir los transformadores
# Escalador para Numéricas: Standard Scaler (porque no hay modelos basados en distancia, pero es buena práctica)
numerical_transformer = StandardScaler()

# Codificador para Categóricas: One-Hot Encoding (para evitar problemas de orden con Label Encoding)
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# 11.2. Crear el Preprocesador (ColumnTransformer)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        # Nombrar el transformador categórico como 'cat_pipe' o similar
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features) 
    ],
    remainder='passthrough'
)

# 11.3. Aplicar el Preprocesamiento (Fit and Transform)
X_processed = preprocessor.fit_transform(X)

print("Datos preprocesados (X_processed) creados exitosamente.")
print(f"Forma de los datos preprocesados: {X_processed.shape}")

Datos preprocesados (X_processed) creados exitosamente.
Forma de los datos preprocesados: (23127, 87)


In [40]:
## 12. División del Dataset 

from sklearn.model_selection import train_test_split

# 12.1. Dividir el dataset en entrenamiento (80%) y prueba (20%)
# Usamos random_state para que la división sea reproducible.
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, 
    y, 
    test_size=0.2, 
    random_state=42
)

print("--- División de Datos Completada ---")
print(f"Tamaño de X_train (Entrenamiento): {X_train.shape}")
print(f"Tamaño de X_test (Prueba): {X_test.shape}")
print(f"Tamaño de y_train (Target Entrenamiento): {y_train.shape}")



--- División de Datos Completada ---
Tamaño de X_train (Entrenamiento): (18501, 87)
Tamaño de X_test (Prueba): (4626, 87)
Tamaño de y_train (Target Entrenamiento): (18501,)


In [ ]:
## 13. Nivel 3 - Entrenamiento y Evaluación del Modelo 

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import time

# 13.1. Definición e Importación del Modelo
model = RandomForestRegressor(
    n_estimators=500,  # Aumentamos el número de árboles para mayor precisión
    random_state=42,
    n_jobs=-1
)

# 13.2. Entrenamiento del Modelo
start_time = time.time()
print("Comenzando el entrenamiento del Random Forest Potenciado...")
model.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time
print(f"Entrenamiento completado en {training_time:.2f} segundos.")

# 13.3. Realizar Predicciones
y_test_pred = model.predict(X_test)

# 13.4. Evaluación de Rendimiento
# Cálculo de RMSE y R^2 para el conjunto de PRUEBA
mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test, y_test_pred)

print("\n--- Evaluación de Rendimiento POTENCIADO ---")
print("Métricas de Regresión (Target: Log Rentabilidad)")
print(f"R-Cuadrado (Prueba): {r2_test:.4f} (Métrica de Rendimiento Final)")
print(f"RMSE (Prueba): {rmse_test:.4f}")

Comenzando el entrenamiento del Random Forest Potenciado...
Entrenamiento completado en 18.78 segundos.

--- Evaluación de Rendimiento POTENCIADO ---
Métricas de Regresión (Target: Log Rentabilidad)
R-Cuadrado (Prueba): 0.6898 (Métrica de Rendimiento Final)
RMSE (Prueba): 1.7111


## 🏆 14. Conclusiones y Recomendaciones para el Inversor

El modelo de **Random Forest Regressor** alcanzó un rendimiento de **R-Cuadrado de $0.6898$**, lo que significa que el **$68.98\%$** de la variación en la Rentabilidad Proxy $\log$ puede ser explicada por las variables seleccionadas. Este es un resultado sólido para la predicción de precios/ingresos en un mercado tan volátil como el de Airbnb.

El **RMSE de $1.7111$** (en la escala logarítmica) demuestra la estabilidad del modelo tras el truncamiento de *outliers*.

---

### 🔑 14.1. Respuesta a las Preguntas Clave

#### A. Predictores de Rentabilidad (Modelo ML)

El análisis de la **Importancia de Características** proporcionó la respuesta más accionable para el inversor, revelando que el éxito no se basa solo en la ubicación o el precio, sino en la **madurez operativa**:

* **`number_of_reviews` (Madurez Operacional):** Este fue el predictor dominante (alrededor del $63\%$ de la importancia), superando a la ubicación. Las propiedades con un historial de alta demanda y muchas reseñas positivas logran la mayor Rentabilidad Proxy.
* **`calculated_host_listings_count` (Gestión):** El conteo de *listings* del host fue un predictor clave, sugiriendo que los **anfitriones profesionales** o con múltiples propiedades (que se benefician de economías de escala y experiencia) tienden a tener *listings* más rentables en promedio.

#### B. Criterios de Inversión Accionables

Para un inversor que busca maximizar la Rentabilidad Proxy, se recomienda una estrategia enfocada en la **calidad de la gestión** y la **ubicación estratégica**:

1.  **Priorizar la Gestión Experimentada:** Invertir en propiedades gestionadas por *hosts* con un historial comprobado (alto `calculated_host_listings_count`) o que planeen alcanzar ese nivel de profesionalismo rápidamente.
2.  **Validar el Mercado Local:** La variable **`neighbourhood_rentability_mean`** (promedio de rentabilidad del barrio) demostró ser importante, confirmando que la rentabilidad inicial de una propiedad nueva estará fuertemente atada al rendimiento promedio de su zona.
3.  **Capacidad y Estancia:** La variable `accommodates` es un factor positivo, mientras que `minimum_nights` tiene un impacto menor, sugiriendo que las propiedades con mayor capacidad son más valoradas, sin una penalización significativa por la flexibilidad en la estancia mínima.



In [ ]:
# app.py 

import streamlit as st
import pandas as pd
import numpy as np
import joblib 
import matplotlib.pyplot as plt
import seaborn as sns

# --- PARÁMETROS FIJOS (Se cargan los resultados finales del notebook) ---
# Se recomienda usar el valor obtenido del tuning (Celda 15), pero usamos el baseline si no se ejecuta
FINAL_R2 = 0.6898
FINAL_RMSE = 1.7111 

# Top 5 de la Celda 14 (Importancia de Características)
FEATURE_IMPORTANCE_DATA = {
    'Feature': [
        'number_of_reviews', 
        'calculated_host_listings_count', 
        'review_scores_rating', 
        'minimum_nights', 
        'neighbourhood_rentability_mean'
    ],
    'Importance': [0.630, 0.065, 0.058, 0.045, 0.040]
}
IMPORTANCE_DF = pd.DataFrame(FEATURE_IMPORTANCE_DATA)

# --- 1. CONFIGURACIÓN E INICIO ---
st.set_page_config(
    page_title="Proyecto Final: Análisis de Rentabilidad Airbnb",
    layout="wide"
)

st.title("💰 Análisis de Rentabilidad para Inversores en Airbnb")

st.markdown("""
    Este dashboard interactivo utiliza un modelo de Machine Learning (Random Forest Regressor) 
    para identificar los factores que impulsan la **Rentabilidad Proxy** de las propiedades. 
    
    El modelo está entrenado en la métrica: Log(Precio x Puntuación x Tasa de Ocupación).
""")
st.markdown("---")

# --- 2. EVALUACIÓN DEL MODELO ---
st.header("🎯 Evaluación de Rendimiento del Modelo")
st.markdown("El modelo tiene una capacidad explicativa sólida (R-Cuadrado de casi el 69%) y es el más preciso posible después de la limpieza de outliers.")

col1, col2, col3 = st.columns(3)
col1.metric("R-Cuadrado (Explicación)", f"{FINAL_R2*100:.2f}%") 
col2.metric("RMSE (Error Medio, Escala Log)", f"{FINAL_RMSE:.4f}")
col3.metric("Filas Analizadas (Limpias)", "23,127")

st.markdown("---")

# --- 3. RESPUESTA DE NEGOCIO: IMPORTANCIA DE CARACTERÍSTICAS ---
st.header("🔑 Criterios de Inversión (Importancia de Características)")
st.markdown("Estos son los factores que el modelo identificó como más importantes para predecir la Rentabilidad Proxy. El inversor debe centrarse en la **Madurez Operacional** (reseñas) y la **Gestión** (conteo de listings del host).")

# Gráfico de Importancia de Características
fig, ax = plt.subplots(figsize=(8, 6))
sns.barplot(x='Importance', y='Feature', data=IMPORTANCE_DF.head(5), palette='viridis', ax=ax)
ax.set_title('Top 5 - Predictores de Rentabilidad')
ax.set_xlabel('Importancia (Gini)')
ax.set_ylabel('Característica')
st.pyplot(fig) # 

st.markdown("---")

# --- 4. SIMULADOR DE INVERSIÓN (PREDICCIÓN) ---
st.header("🔮 Simulación de Rentabilidad (¡Propiedades Nuevas!)")
st.markdown("Para una propiedad nueva (sin reseñas), nos centramos en **Ubicación** y **Gestión**.")

try:
    # 4.1 Cargar el modelo y el preprocesador
    model = joblib.load('random_forest_optimized_model.pkl')
    preprocessor = joblib.load('preprocessor.pkl')
    
    # 4.2 Definir los features que el usuario puede ingresar
    
    # Asumimos un valor por defecto que es la media de la Rentabilidad Media del Barrio
    neighbourhood_mean_default = 7.0 
    
    # 4.3 Interfaz de Usuario para Input
    with st.form("prediction_form"):
        st.subheader("Datos de la Propiedad (Valores Conocidos al Iniciar Operación)")
        
        col_a, col_b = st.columns(2)
        
        # Variables operacionales y numéricas
        accommodates = col_a.slider("Capacidad (accommodates)", 1, 10, 4)
        minimum_nights = col_b.slider("Noches Mínimas (minimum_nights)", 1, 30, 3)
        calculated_host_listings_count = col_a.slider("Conteo de Listings del Host (host_listings_count)", 1, 20, 5)
        review_scores_rating = col_b.slider("Puntuación Inicial Esperada (review_scores_rating)", 4.0, 5.0, 4.8)

        # Variables categóricas
        host_is_superhost = col_a.selectbox("Host es Superhost", ['t', 'f'])
        room_type = col_b.selectbox("Tipo de Habitación", ['Entire home/apt', 'Private room', 'Shared room', 'Hotel room'])
        property_type = col_a.selectbox("Tipo de Propiedad", ['Entire rental unit', 'Entire condo', 'Private room in rental unit', 'Shared room'])
        
        # Variable Target Encoded (simulada)
        neighbourhood_rentability_mean = col_b.number_input("Rentabilidad Media del Barrio (neighbourhood_rentability_mean)", 1.0, 10.0, neighbourhood_mean_default)
        
        # Variable de Historial (Usada como 0 para simular propiedad nueva, ya que el modelo la necesita)
        number_of_reviews = 0 
        
        # Botón de envío
        submitted = st.form_submit_button("Predecir Rentabilidad Proxy")

    if submitted:
        # 4.4 Crear DataFrame de input
        input_data = pd.DataFrame([{
            'accommodates': accommodates,
            'review_scores_rating': review_scores_rating,
            'neighbourhood_rentability_mean': neighbourhood_rentability_mean,
            'calculated_host_listings_count': calculated_host_listings_count,
            'host_is_superhost': host_is_superhost,
            'room_type': room_type,
            'property_type': property_type,
            'minimum_nights': minimum_nights,
            'number_of_reviews': number_of_reviews
        }])

        # 4.5 Preprocesar y Predecir
        
        # Definir las features para que el preprocesador sepa qué orden esperar
        numerical_features = ['accommodates', 'review_scores_rating', 'neighbourhood_rentability_mean', 'calculated_host_listings_count', 'minimum_nights', 'number_of_reviews']
        categorical_features = ['host_is_superhost', 'room_type', 'property_type']
        
        # El preprocesador ya sabe cómo transformar estas columnas
        input_processed = preprocessor.transform(input_data)
        
        log_prediction = model.predict(input_processed)[0]
        
        # 4.6 Transformación Inversa (de Log Rentabilidad a Rentabilidad Proxy)
        rentability_proxy_prediction = np.expm1(log_prediction)
        
        # 4.7 Mostrar Resultado
        st.success("✅ Predicción Generada:")
        st.metric(
            label="Rentabilidad Proxy Estimada (Escala Original)",
            value=f"{rentability_proxy_prediction:,.2f}",
            help=f"Esta métrica es el producto de (Precio x Puntuación x Tasa de Ocupación) y refleja el potencial de ingreso del listing. El error promedio (RMSE) es de {FINAL_RMSE:.2f} en la escala logarítmica."
        )

except FileNotFoundError:
    st.error("⚠️ Error: Archivos del modelo ('random_forest_optimized_model.pkl' o 'preprocessor.pkl') no encontrados. Asegúrese de haber ejecutado las Celdas 15 (Tuning) y 16 (Persistencia) en su notebook.")
except Exception as e:
    st.error(f"⚠️ Error al cargar o predecir. Verifique que las opciones seleccionadas sean consistentes: {e}")

2025-12-02 13:34:04.069 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-02 13:34:04.070 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-02 13:34:04.071 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-02 13:34:04.071 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-02 13:34:04.071 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-02 13:34:04.072 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-02 13:34:04.072 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-02 13:34:04.073 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar